In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, ICRS, Galactic, GalacticLSR, Galactocentric
import astropy.units as u
import galpy
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014, vcirc
from galpy.potential import NFWPotential, HernquistPotential, MiyamotoNagaiPotential, NonInertialFrameForce
import warnings
warnings.filterwarnings("ignore")

import movie_stars

In [2]:
def compute_median_gal_motions(df):
    icrs = ICRS(ra=df.ra.values * u.deg,
                dec=df.dec.values * u.deg,
                distance=df.distance.values * u.pc,
                pm_ra_cosdec=df.pmra_real.values * (u.mas / u.yr),
                pm_dec=df.pmdec_real.values * (u.mas / u.yr),
                radial_velocity=df.rv_real.values * (u.km / u.s))
    gal = icrs.transform_to(Galactic)
    gal.representation_type = 'cartesian'
    gal.differential_type = 'cartesian'
    gal = Galactic(gal.u,
                   gal.v,
                   gal.w, [np.nanmedian(gal.U)] * len(df),
                   [np.nanmedian(gal.V)] * len(df),
                   [np.nanmedian(gal.W)] * len(df),
                   representation_type='cartesian',
                   differential_type='cartesian')
    gal.representation_type = 'spherical'
    gal.differential_type = 'spherical'
    icrs = gal.transform_to(ICRS)

    df['pmra_med'] = icrs.pm_ra_cosdec.value
    df['pmdec_med'] = icrs.pm_dec.value
    df['rv_med'] = icrs.radial_velocity.value

    return df


def compute_circ_orbits(df):
    icrs = ICRS(ra=df.ra.values * u.deg,
                dec=df.dec.values * u.deg,
                distance=df.distance.values * u.pc,
                pm_ra_cosdec=df.pmra_real.values * (u.mas / u.yr),
                pm_dec=df.pmdec_real.values * (u.mas / u.yr),
                radial_velocity=df.rv_real.values * (u.km / u.s))
    gc = icrs.transform_to(Galactocentric)
    gc.representation_type = 'cylindrical'
    gc.differential_type = 'cylindrical'

    mp = MiyamotoNagaiPotential()
    hp = HernquistPotential()
    nfp = NFWPotential()

    rho = (-1) * gc.rho
    rho_scaled = rho / (8000)
    #v_circ = mp.vcirc(R=-gc.rho, phi=gc.phi) + hp.vcirc(R=-gc.rho, phi=gc.phi) + nfp.vcirc(R=-gc.rho, phi=gc.phi)
    v_circ = vcirc(MWPotential2014, R=-gc.rho, phi=gc.phi)
    d_phi_corrected = (v_circ * 220 * u.km/u.s / rho).to(u.mas/u.yr, u.dimensionless_angles())

    gc_new = Galactocentric(rho=gc.rho,
                            phi=gc.phi,
                            z=gc.z,
                            d_rho=np.zeros(len(gc))*((u.mas*u.pc)/(u.rad*u.yr)),
                            d_phi=d_phi_corrected,
                            d_z=np.zeros(len(gc))*((u.mas*u.pc)/(u.rad*u.yr)),
                            representation_type='cylindrical',
                            differential_type='cylindrical')
    gc_new.representation_type = 'spherical'
    gc_new.differential_type = 'spherical'

    icrs = gc_new.transform_to(ICRS)
    df['pmra_med'] = icrs.pm_ra_cosdec.value
    df['pmdec_med'] = icrs.pm_dec.value
    df['rv_med'] = icrs.radial_velocity.value

    return df


def sub_no_pec_motion(df, time_value):
    df = df.loc[np.round(df.t, 1) == time_value]
    df['pmra_real'] = df['pmra']
    df['pmdec_real'] = df['pmdec']
    df['rv_real'] = df['rv']

    df = df.drop(columns=['pmra', 'pmdec', 'rv'])
    #df = compute_median_gal_motions(df)
    df = compute_circ_orbits(df)
    df['parallax'] = 1000 / df.distance
    # df['pmra_med'] = [df.pmra_real.median()] * len(df)
    # df['pmdec_med'] = [df.pmdec_real.median()] * len(df)
    # df['rv_med'] = [df.rv_real.median()] * len(df)

    return df

In [3]:
#df_expanding_clusters_integrated = pd.read_csv('../maria_feedback_clusters_integrated.csv')
df_expanding_clusters_integrated = pd.read_csv('../clusters_integrated/maria_expanding_clusters_1_int.csv')
#df_expanding_clusters_integrated = pd.read_csv('../clusters_integrated/maria_heiles_clusters_int.csv')
df_expanding_clusters = sub_no_pec_motion(df_expanding_clusters_integrated, -40.6)

# Simulating Expanding Clusters

In [5]:
time_to_integrate = np.linspace(0, 60, 100)
maria_marker_props_fb = {
    'size': 5.,
    'color': 'cyan',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
key_dict = {'ra':'ra', 'dec':'dec', 'parallax':'parallax', 'pmra_med':'pmra', 'pmdec_med': 'pmdec', 'rv_med':'radial_velocity', 'group_name': 'group_name', 'age':'age'}
maria_clusters_no_pec = movie_stars.MovieStar(
    name='Clusters without peculiar motions',
    df=df_expanding_clusters,
    key_dict=key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_fb,
    marker_behavior='remain',
    visible=True)

    

maria_marker_props_fb = {
    'size': 10.,
    'color': 'red',
    'opacity': .7,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
key_dict2 = {'ra':'ra', 'dec':'dec', 'parallax':'parallax', 'pmra_real':'pmra', 'pmdec_real': 'pmdec', 'rv_real':'radial_velocity', 'group_name': 'group_name', 'age':'age'}
maria_clusters_expanding= movie_stars.MovieStar(
    name='Clusters with peculiar motions',
    df=df_expanding_clusters,
    key_dict=key_dict2,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_fb,
    marker_behavior='remain',
    visible=True)

traces_list = [maria_clusters_no_pec, maria_clusters_expanding]
movie = movie_stars.Movie(movie_stars=traces_list,
                          time=time_to_integrate,
                          movie_save_path='/Users/cam/Downloads/clusters_circ_motion_1.html',
                          plot_sun = False,
                          center_star='Clusters without peculiar motions',
                          annotations=None,
                          xyz_ranges=[4000, 4000, 700],
                          camera_follow=True,
                          center_galactic=False,
                          add_extra_traces=True)
movie.make_movie()

Movie Created


# Simulating Radcliffe Wave

In [19]:
df_rw_integrated = pd.read_csv('../clusters_integrated/rw_int.csv')
df_rw = sub_no_pec_motion(df_rw_integrated, 0.)

In [20]:
time_to_integrate = np.linspace(0, 60, 100)
rw_marker_props = {
    'size': 5.,
    'color': 'cyan',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra_med': 'pmra',
    'pmdec_med': 'pmdec',
    'rv_med': 'radial_velocity',
    'group_name': 'group_name',
    'age': 'age'
}
rw_no_pec = movie_stars.MovieStar(name='RW without peculiar motions',
                                  df=df_rw,
                                  key_dict=key_dict,
                                  time_to_integrate=time_to_integrate,
                                  marker_properties=rw_marker_props,
                                  marker_behavior='remain',
                                  visible=True)




rw_marker_props2 = {
    'size': 10.,
    'color': 'red',
    'opacity': .7,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
key_dict2 = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra_real': 'pmra',
    'pmdec_real': 'pmdec',
    'rv_real': 'radial_velocity',
    'group_name': 'group_name',
    'age': 'age'
}
rw = movie_stars.MovieStar(name='RW real motions',
                                  df=df_rw,
                                  key_dict=key_dict2,
                                  time_to_integrate=time_to_integrate,
                                  marker_properties=rw_marker_props2,
                                  marker_behavior='remain',
                                  visible=True)

traces_list = [rw, rw_no_pec]
movie = movie_stars.Movie(
    movie_stars=traces_list,
    time=time_to_integrate,
    movie_save_path='/Users/cam/Downloads/rw_no_pec_motion.html',
    plot_sun=False,
    center_star='RW real motions',
    annotations=None,
    xyz_ranges=[4000, 4000, 500],
    camera_follow=True,
    center_galactic=False,
    add_extra_traces=True)
movie.make_movie()


Movie Created
